In [1]:
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
lumi=3000

# Remove empty sublists within each list
def clean_array(array):
    return np.array(np.squeeze(ak.Array([[sublist for sublist in inner_list if len(sublist) > 0] for inner_list in array])))
                    

In [11]:
data = ak.from_parquet("hnl_test2.parquet")
events_to_inject = len(data)
in_fid = np.all(np.array(data.in_fiducial),axis=-1)
muon_flag = np.abs(data.secondary_types) == 13
hnl_flag = np.abs(data.primary_type) == 5914
mu_vertex = clean_array(data.vertex[hnl_flag])
muon_momenta = clean_array(data.secondary_momenta[muon_flag])
muon_momentum = np.linalg.norm(muon_momenta[:,:,1:],axis=-1)
mu_dir = muon_momenta[:,:,1:] / np.expand_dims(muon_momentum,-1)

In [16]:
np.dot(mu_dir[:,0,:],mu_dir[:,1,:])

TypeError: dot() got an unexpected keyword argument 'axis'

In [4]:
len(data.weights[in_fid])

139

In [7]:
data.se

<Array [{event_weight: 1e-48, ...}, ..., {...}] type='1000 * {event_weight:...'>

In [8]:
data = ak.from_parquet("Data/SIREN/Output/UNDINE_LHCb_North/LHC13_SIBYLL_light_14_NC.parquet")


KeyboardInterrupt



In [61]:
data.weights[in_fid]

<Array [0.0697, 0.03, 0.117, ..., 0.0928, 0.0212] type='138398 * float64'>

In [62]:
sum(data.weights[in_fid])#int_probs[data.in_fiducial]

35929.91449331461

In [2]:
import siren
import os
# Now include DIS interaction
cross_section_model = "HNLDISSplines"

xsfiledir = siren.utilities.get_cross_section_model_path(cross_section_model)

# Cross Section Model
target_type = siren.dataclasses.Particle.ParticleType.Nucleon

nutype = "nu"
primary = 14
primary_type = (siren.dataclasses.Particle.ParticleType)(primary)

m4 = "0000300"
Ue4 = 0
Umu4 = 0.01
Utau4 = 0

HNL_DIS_xs = siren.interactions.HNLDISFromSpline(
    os.path.join(xsfiledir, "M_0000000MeV/dsdxdy-%s-N-nc-GRV98lo_patched_central.fits"%nutype),
    os.path.join(xsfiledir, "M_%sMeV/sigma-%s-N-nc-GRV98lo_patched_central.fits"%(m4,nutype)),
    float(m4)*1e-3,
    [Ue4,Umu4,Utau4],
    siren.utilities.Constants.isoscalarMass,
    1,
    [primary_type],
    [target_type],
)
xsfiledir = "/n/holylfs05/LABS/arguelles_delgado_lab/Everyone/pweigel/cross_sections/20241017"

# Cross Section Model
target_type = siren.dataclasses.Particle.ParticleType.Nucleon
nutype = "neutrino"
nuflavor = "muon"
xs_mode = "NC"
minQ2 = 1

DIS_xs = siren.interactions.DISFromSpline(
        os.path.join(xsfiledir, "wcg24b_dsdxdy_%s_%s_%s_isoscalar.fits"%(xs_mode,nuflavor,nutype)),
        os.path.join(xsfiledir, "wcg24b_sigma_%s_%s_%s_isoscalar.fits"%(xs_mode,nuflavor,nutype)),
        1,siren.utilities.Constants.isoscalarMass,minQ2,
        [primary_type],
        [target_type], "cm"
    )

In [3]:

DIS_xs.TotalCrossSection(primary_type,1000) # cm^2

1.9182853408097895e-36

In [4]:
HNL_DIS_xs.TotalCrossSection(primary_type,1000)# * 1e18 * siren.utilities.Constants.hbarc**2 * 1e4 # cm^2

1.9821680062720344e-40